In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv # add this line

In [ ]:
data = 'data-sample_data-nyctaxi-trips-2009-json_corrigido.json'

In [ ]:
df = pd.read_json(data, lines=True)

In [ ]:
df.to_csv('data-sample_data-nyctaxi-trips-2009.csv', index=False)

In [ ]:
df_csv = pd.read_csv('data-sample_data-nyctaxi-trips-2009.csv')

In [ ]:
df_csv.head()

In [ ]:
df_csv.dtypes

In [ ]:
# converter datatype de object para datetime
df_csv['pickup_datetime'] = pd.to_datetime(df_csv['pickup_datetime'], format='mixed').dt.tz_convert(None)
df_csv['dropoff_datetime'] = pd.to_datetime(df_csv['dropoff_datetime'], format='mixed').dt.tz_convert(None)

In [ ]:
df_csv.dtypes

In [ ]:
# formatar datetime
df_csv['pickup_datetime'] = pd.to_datetime(df_csv['pickup_datetime'], format='Y%m%d %H:%M:%S')
df_csv['dropoff_datetime'] = pd.to_datetime(df_csv['dropoff_datetime'], format='Y%m%d %H:%M:%S')

In [ ]:
df_csv.head(2)

In [ ]:
df_csv.to_parquet("../data/data-sample_data-nyctaxi-trips-2009-json_corrigido.parquet", compression='gzip')

In [ ]:
load_dotenv() # add this line
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
db = os.getenv('MYSQL_DATABASE')
port = os.getenv('MYSQL_PORT')

In [ ]:
try:
    conn = pymysql.connect(host = host,
                           port = int(port),
                           user = user,
                           password = password,
                           db = db)
except e:
    print (e)
if conn:
    print ("connection successful")
else:
        print ("error")

In [ ]:
print(pd.io.sql.get_schema(df, name='nyctaxi-trips', con=conn))

In [ ]:
df_csv_iter = pd.read_csv('../data/data-sample_data-nyctaxi-trips-2009.csv', iterator=True, chunksize=100000)

In [ ]:
df_csv = next(df_csv_iter)

In [ ]:
len(df_csv)

In [ ]:
df_csv.head(n=0).to_sql(name='nyctaxi-trips', con=engine, if_exists='replace')

In [ ]:
%time df_csv.to_sql(name='nyctaxi-trips', con=engine, if_exists='append')

In [ ]:
from time import time

In [ ]:
while True:

    try:
        t_start = time()

        df = next(df_iter)

        df_csv['pickup_datetime'] = pd.to_datetime(df_csv['pickup_datetime'], format='mixed')
        df_csv['dropoff_datetime'] = pd.to_datetime(df_csv['dropoff_datetime'], format='mixed')

        df_csv.to_sql(name=table_name, con=engine, if_exists='append')

        t_end = time()

        print('inserted another chunk, took %.3f second' % (t_end - t_start))

    except StopIteration:
        print("Finished ingesting data into the postgres database")
        break
